In [21]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from openai import OpenAI
from tqdm.auto import tqdm
import torch
import pickle

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [2]:
from project_secrets import GPT_4o_API_KEY

In [3]:
GPT_4o_MODEL = "openai/gpt-4o-mini"
client_4o = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=GPT_4o_API_KEY)

In [4]:
ds = load_dataset("ruslanmv/ai-medical-chatbot")

In [5]:
ds["train"][0]["Description"]

'Q. What does abutment of the nerve root mean?'

In [6]:
qa_pairs = [(entry["Patient"], entry["Doctor"]) for entry in ds["train"]]

In [7]:
questions, answers = zip(*qa_pairs)

In [19]:
embed_model = SentenceTransformer("paraphrase-MiniLM-L3-v2")  # Fast and effective

In [20]:
# Process in batches
batch_size = 32
question_embeddings = []

for i in tqdm(range(0, len(questions), batch_size)):
    batch = questions[i:i + batch_size]
    batch_embeddings = embed_model.encode(batch, convert_to_numpy=True, show_progress_bar=True)
    question_embeddings.append(batch_embeddings)

question_embeddings = np.vstack(question_embeddings)

100%|██████████| 8029/8029 [1:42:40<00:00,  1.30it/s]


In [22]:
# ! Only run this cell when you want to save question_embeddings
with open("question_embeddings.pkl", "wb") as f:
    pickle.dump(question_embeddings, f)

In [ ]:
# ! Only run this cell when you want to LOAD question_embeddings
with open("question_embeddings.pkl", "rb") as f:
    question_embeddings = pickle.load(f)

In [24]:
# Store embeddings in FAISS for fast retrieval
dimension = question_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(question_embeddings)

In [25]:
def retrieve_context(user_query, top_k=3):
    """Finds the most relevant stored questions and their answers"""
    query_embedding = embed_model.encode([user_query], convert_to_numpy=True)
    _, indices = index.search(query_embedding, top_k)
    
    retrieved_context = "\n\n".join([f"Patient: {questions[i]}\nDoctor: {answers[i]}" for i in indices[0]])
    return retrieved_context

In [26]:
def medical_chatbot(user_query):
    """Uses OpenAI's GPT-4 to generate a response with retrieved context"""
    retrieved_info = retrieve_context(user_query)
    prompt = f"""
    You are a helpful and professional medical chatbot. Below is past conversation data:

    {retrieved_info}

    Now answer the following question in a helpful and concise manner:
    Patient: {user_query}
    Doctor:
    """
    response = client_4o.chat.completions.create(
        model=GPT_4o_MODEL,
        messages=[
            {
                "role": "system", "content": "You are a medical chatbot."
            },
            {
                "role": "user", "content": prompt
            }
        ],
        temperature=0.7
    )
    return response.choices[0].message.content

In [28]:
user_query = "I have nerve root pain, what should I do?"
ai_response = medical_chatbot(user_query)
print("Chatbot:", ai_response)

Chatbot: Hello, 

If you are experiencing nerve root pain, it's important to take the following steps:

1. **Rest**: Try to avoid activities that exacerbate the pain. Short periods of bed rest can be beneficial initially.

2. **Pain Relief**: Over-the-counter pain medications such as ibuprofen or acetaminophen can help relieve pain and inflammation.

3. **Physical Therapy**: Consider consulting a physical therapist who can provide exercises to strengthen your core and back muscles, which can help alleviate pressure on the nerve.

4. **Heat/Ice Therapy**: Applying heat or ice to the affected area may provide relief.

5. **Avoid Aggravating Activities**: Avoid heavy lifting, twisting motions, and prolonged sitting or standing. 

6. **Medical Consultation**: If the pain persists or worsens, it is advisable to consult a healthcare professional, such as a neurologist or orthopedic surgeon, for further evaluation and treatment options.

Please let me know if you have any further questions or